In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
df = pd.read_csv('heart-2020-cleaned-numerical.csv')


In [3]:
print(len(df))

319795


In [4]:
df = df.replace('Yes', 1, regex=True).replace('No', 0, regex=True).replace('Yes (during pregnancy)', 1, regex=True)
df = df.replace('Female', 0, regex=True).replace('Male', 1, regex=True)
df = df.replace('18-24', 0, regex=True).replace("25-29", 1, regex=True).replace("30-34", 2, regex=True).replace("35-39", 3, regex=True)
df = df.replace("40-44", 4, regex=True).replace("45-49", 5, regex=True).replace("50-54", 6, regex=True).replace("55-59", 7, regex=True)
df = df.replace("60-64", 8, regex=True).replace("65-69", 9, regex=True).replace("70-74", 10, regex=True).replace("75-79", 11, regex=True).replace("80 or older", 12, regex=True)

df =df.replace("American Indian/Alaskan Native", 0, regex=True)
df =df.replace("Asian", 1, regex=True)
df =df.replace("Black", 2, regex=True)
df =df.replace("Hispanic", 3, regex=True)
df =df.replace("White", 4, regex=True)
df =df.replace("Other", 5, regex=True)



df.head(5)

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,0,16.60,1,0,0,3.0,30.0,0,0,7,4,1,1,4,5.0,1,0,1
1,0,20.34,0,0,1,0.0,0.0,0,0,12,4,0,1,4,7.0,0,0,0
2,0,26.58,1,0,0,20.0,30.0,0,1,9,4,1,1,2,8.0,1,0,0
3,0,24.21,0,0,0,0.0,0.0,0,0,11,4,0,0,3,6.0,0,0,1
4,0,23.71,0,0,0,28.0,0.0,1,0,4,4,0,1,4,8.0,0,0,0


In [5]:
print("total" ,len(df))

total 319795


In [6]:
# balancing data
# seperate by col value
print("total" ,len(df))
df_HD = df.loc[df['HeartDisease'] == 1]
df_NHD = df.loc[df['HeartDisease'] == 0]
print('hd', len(df_HD))
print('nhd', len(df_NHD))
#get equal amount of sample of each
df_HD = df_HD.sample(27373)
print('curr total', len(df_HD))
df_NHD = df_NHD.sample(27373)
print('curr total', len(df_NHD))
# merge the two types
df = pd.concat([df_HD, df_NHD], axis=0)
print(len(df))
# df.head(3)

total 319795
hd 27373
nhd 292422
curr total 27373
curr total 27373
54746


In [7]:
X = df.loc[:, df.columns != 'HeartDisease']
y = df['HeartDisease']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)
print('training size: ', len(X_train), len(y_train))
print('test size: ', len(X_test), len(y_test))

training size:  43796 43796
test size:  10950 10950


In [8]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import GradientBoostingClassifier
gbclf = GradientBoostingClassifier()
cv_results = cross_validate(gbclf, X_train, y_train, cv=5)
print('gradient booster test score: ', sum(cv_results['test_score'])/len(cv_results['test_score']))#gives 85
from sklearn.model_selection import GridSearchCV
import numpy as np

gradient booster test score:  0.762923490046246


In [11]:

parameters = {
    'learning_rate' : np.arange(0, 1.1, 0.1), 
    'n_estimators': np.arange(100, 501, 100)
    # 'subsample': np.arange(0, 1.1, 0.1)
}
# n_cpus = multiprocessing.cpu_count()
gbc = GridSearchCV(gbclf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
gbc.fit(X_train,y_train)

Fitting 5 folds for each of 55 candidates, totalling 275 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 275 out of 275 | elapsed:  7.0min finished


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
                         'n_estimators': array([100, 200, 300, 400, 500])},
             scoring='f1', verbose=1)

In [12]:

print ("Best parameters: " , str(gbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(gbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'learning_rate': 0.1, 'n_estimators': 100}
Average score from cross-validation of best parameters on training data:  0.7766512699469206
F1 score of test data: 0.7739221871713985


In [13]:

parameters = {
    'learning_rate' : np.arange(0, 0.3, 0.1), 
    'n_estimators': np.arange(200, 400, 50),
    'subsample': np.arange(0, 1.1, 0.1)
}
# n_cpus = multiprocessing.cpu_count()
gbc = GridSearchCV(gbclf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
gbc.fit(X_train,y_train)

Fitting 5 folds for each of 132 candidates, totalling 660 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 470 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 660 out of 660 | elapsed:  7.3min finished


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': array([0. , 0.1, 0.2]),
                         'n_estimators': array([200, 250, 300, 350]),
                         'subsample': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])},
             scoring='f1', verbose=1)

In [14]:

print ("Best parameters: " , str(gbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(gbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'learning_rate': 0.1, 'n_estimators': 200, 'subsample': 0.7000000000000001}
Average score from cross-validation of best parameters on training data:  0.7763767949074396
F1 score of test data: 0.7737917726515218


In [15]:

parameters = {
    'loss' : ['log_loss', 'deviance', 'exponential'], 
    'n_estimators': np.arange(100, 400, 50),
    'subsample': np.arange(0.7, 1.0, 0.1),
    'criterion': ['friedman_mse', 'squared_error', 'mse']
}
# n_cpus = multiprocessing.cpu_count()
gbc = GridSearchCV(gbclf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
gbc.fit(X_train,y_train)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 1000 tasks      | elapsed: 18.9min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed: 24.7min finished


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'criterion': ['friedman_mse', 'squared_error', 'mse'],
                         'loss': ['log_loss', 'deviance', 'exponential'],
                         'n_estimators': array([100, 150, 200, 250, 300, 350]),
                         'subsample': array([0.7, 0.8, 0.9, 1. ])},
             scoring='f1', verbose=1)

In [16]:

print ("Best parameters: " , str(gbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(gbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'criterion': 'friedman_mse', 'loss': 'deviance', 'n_estimators': 100, 'subsample': 0.7}
Average score from cross-validation of best parameters on training data:  0.7768281624962654
F1 score of test data: 0.7735038990624726


In [17]:

parameters = {
    'loss' : ['deviance'], 
    'n_estimators': np.arange(150, 260, 25),
    'subsample': [0.7, 0.8, 0.9],
    'criterion': ['friedman_mse'],
    'min_samples_split': np.arange(0.0, 1.1, 0.1),
    'min_samples_leaf': np.arange(0.0, 1.1, 0.1),
}
# n_cpus = multiprocessing.cpu_count()
gbc = GridSearchCV(gbclf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
gbc.fit(X_train,y_train)

Fitting 5 folds for each of 1815 candidates, totalling 9075 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 1072 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1350 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 1700 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 2230 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 2860 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 3590 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 4420 tasks      | elapsed: 11.5min
[Parallel(n_jobs=-1)]: Done 8790 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done 9075 out of 9075 | elapsed: 12.4min finished


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'criterion': ['friedman_mse'], 'loss': ['deviance'],
                         'min_samples_leaf': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
                         'min_samples_split': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
                         'n_estimators': array([150, 175, 200, 225, 250]),
                         'subsample': [0.7, 0.8, 0.9]},
             scoring='f1', verbose=1)

In [18]:

print ("Best parameters: " , str(gbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(gbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'criterion': 'friedman_mse', 'loss': 'deviance', 'min_samples_leaf': 0.1, 'min_samples_split': 0.1, 'n_estimators': 250, 'subsample': 0.9}
Average score from cross-validation of best parameters on training data:  0.7718139969962274
F1 score of test data: 0.7675050389974586


In [19]:

parameters = {
    'loss' : ['deviance'], 
    'n_estimators': np.arange(100, 260, 50),
    'subsample': [0.8],
    'criterion': ['friedman_mse'],
    'min_samples_split': [0.1, 0.2, 0.3, 2],
    'min_samples_leaf': [0, 0.1, 0.2, 1],
    'min_weight_fraction_leaf': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
#     'max_depth': np.arange(1, 16, 2),
#     'min_impurity_decrease': np.arange(0, 1000, 300),
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'ccp_alpha': [0.0, 0.01, 0.1, 0.075, 10, 100]
}
# n_cpus = multiprocessing.cpu_count()
gbc = GridSearchCV(gbclf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
gbc.fit(X_train,y_train)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 750 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1000 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 1350 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 1800 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 1920 out of 1920 | elapsed:  7.0min finished


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'criterion': ['friedman_mse'], 'loss': ['deviance'],
                         'min_samples_leaf': [0, 0.1, 0.2, 1],
                         'min_samples_split': [0.1, 0.2, 0.3, 2],
                         'min_weight_fraction_leaf': [0.0, 0.1, 0.2, 0.3, 0.4,
                                                      0.5],
                         'n_estimators': array([100, 150, 200, 250]),
                         'subsample': [0.8]},
             scoring='f1', verbose=1)

In [20]:

print ("Best parameters: " , str(gbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(gbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'criterion': 'friedman_mse', 'loss': 'deviance', 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 150, 'subsample': 0.8}
Average score from cross-validation of best parameters on training data:  0.7767914379299381
F1 score of test data: 0.7745847614025836


In [21]:

parameters = {
    'loss' : ['deviance'], 
    'n_estimators':np.arange(100, 311, 50),
    'subsample': [0.8],
    'criterion': ['friedman_mse'],
    'min_samples_split': [0.05, 0.1, 0.15],
    'min_samples_leaf': [1],
    'min_weight_fraction_leaf': [0.0, 0.01],
    'max_depth': np.arange(1, 16, 2),
#     'min_impurity_decrease': np.arange(0, 1000, 300),
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'ccp_alpha': [0.0, 0.01, 0.1, 0.075, 10, 100]
}
# n_cpus = multiprocessing.cpu_count()
gbc = GridSearchCV(gbclf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
gbc.fit(X_train,y_train)

Fitting 5 folds for each of 240 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:   53.0s
[Parallel(n_jobs=-1)]: Done 430 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 780 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed: 16.7min finished


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'criterion': ['friedman_mse'], 'loss': ['deviance'],
                         'max_depth': array([ 1,  3,  5,  7,  9, 11, 13, 15]),
                         'min_samples_leaf': [1],
                         'min_samples_split': [0.05, 0.1, 0.15],
                         'min_weight_fraction_leaf': [0.0, 0.01],
                         'n_estimators': array([100, 150, 200, 250, 300]),
                         'subsample': [0.8]},
             scoring='f1', verbose=1)

In [22]:

print ("Best parameters: " , str(gbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(gbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'criterion': 'friedman_mse', 'loss': 'deviance', 'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 0.1, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'subsample': 0.8}
Average score from cross-validation of best parameters on training data:  0.777770047311337
F1 score of test data: 0.7747968190159924


In [23]:

parameters = {
    'loss' : ['deviance'], 
    'n_estimators':[100, 110, 90],
    'subsample': [0.8],
    'criterion': ['friedman_mse'],
    'min_samples_split': [0.025, 0.05, 0.075],
    'min_samples_leaf': [1],
    'min_weight_fraction_leaf': [0.0, 0.001],
    'max_depth': [4, 5, 6],
    'min_impurity_decrease': np.arange(0, 1000, 300),
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'ccp_alpha': [0.0, 0.01, 0.1, 0.075, 10, 100]
}
# n_cpus = multiprocessing.cpu_count()
gbc = GridSearchCV(gbclf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
gbc.fit(X_train,y_train)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:   40.3s
[Parallel(n_jobs=-1)]: Done 430 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 780 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:  3.4min finished


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'criterion': ['friedman_mse'], 'loss': ['deviance'],
                         'max_depth': [4, 5, 6],
                         'min_impurity_decrease': array([  0, 300, 600, 900]),
                         'min_samples_leaf': [1],
                         'min_samples_split': [0.025, 0.05, 0.075],
                         'min_weight_fraction_leaf': [0.0, 0.001],
                         'n_estimators': [100, 110, 90], 'subsample': [0.8]},
             scoring='f1', verbose=1)

In [24]:

print ("Best parameters: " , str(gbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(gbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'criterion': 'friedman_mse', 'loss': 'deviance', 'max_depth': 5, 'min_impurity_decrease': 0, 'min_samples_leaf': 1, 'min_samples_split': 0.025, 'min_weight_fraction_leaf': 0.001, 'n_estimators': 100, 'subsample': 0.8}
Average score from cross-validation of best parameters on training data:  0.7780012788281134
F1 score of test data: 0.7743067743067743


In [25]:

parameters = {
    'loss' : ['deviance'], 
    'n_estimators':[100],
    'subsample': [0.8],
    'criterion': ['friedman_mse'],
    'min_samples_split': [0.05],
    'min_samples_leaf': [1],
    'min_weight_fraction_leaf': [0.0, 0.001, 0.005],
    'max_depth': [5, 6, 7],
    'min_impurity_decrease': np.arange(0, 100, 50),
    'max_features': ['auto', 'sqrt', 'log2'],
    'ccp_alpha': [0.0, 0.01, 0.1, 0.075, 10, 100]
}
# n_cpus = multiprocessing.cpu_count()
gbc = GridSearchCV(gbclf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
gbc.fit(X_train,y_train)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:   31.7s
[Parallel(n_jobs=-1)]: Done 430 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 780 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 1230 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 1620 out of 1620 | elapsed:  5.2min finished


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'ccp_alpha': [0.0, 0.01, 0.1, 0.075, 10, 100],
                         'criterion': ['friedman_mse'], 'loss': ['deviance'],
                         'max_depth': [5, 6, 7],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_impurity_decrease': array([ 0, 50]),
                         'min_samples_leaf': [1], 'min_samples_split': [0.05],
                         'min_weight_fraction_leaf': [0.0, 0.001, 0.005],
                         'n_estimators': [100], 'subsample': [0.8]},
             scoring='f1', verbose=1)

In [26]:

print ("Best parameters: " , str(gbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(gbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'loss': 'deviance', 'max_depth': 6, 'max_features': 'sqrt', 'min_impurity_decrease': 0, 'min_samples_leaf': 1, 'min_samples_split': 0.05, 'min_weight_fraction_leaf': 0.001, 'n_estimators': 100, 'subsample': 0.8}
Average score from cross-validation of best parameters on training data:  0.7785364380365123
F1 score of test data: 0.7734128376008418


In [27]:

parameters = {
    'learning_rate': [0.025, 0.1, 0.12],
    'loss' : ['deviance'], 
    'n_estimators':[100],
    'subsample': [0.8],
    'criterion': ['friedman_mse'],
    'min_samples_split': [0.05],
    'min_samples_leaf': [1],
    'min_weight_fraction_leaf': [0.001, 0.002],
    'max_depth': [6],
    'min_impurity_decrease': np.arange(0, 50, 5),
    'max_features': ['sqrt'],
    'ccp_alpha': [0.0, 0.001, 1000]
}
# n_cpus = multiprocessing.cpu_count()
gbc = GridSearchCV(gbclf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
gbc.fit(X_train,y_train)

Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:   21.8s
[Parallel(n_jobs=-1)]: Done 430 tasks      | elapsed:   50.3s
[Parallel(n_jobs=-1)]: Done 780 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:  1.8min finished


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'ccp_alpha': [0.0, 0.001, 1000],
                         'criterion': ['friedman_mse'],
                         'learning_rate': [0.025, 0.1, 0.12],
                         'loss': ['deviance'], 'max_depth': [6],
                         'max_features': ['sqrt'],
                         'min_impurity_decrease': array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45]),
                         'min_samples_leaf': [1], 'min_samples_split': [0.05],
                         'min_weight_fraction_leaf': [0.001, 0.002],
                         'n_estimators': [100], 'subsample': [0.8]},
             scoring='f1', verbose=1)

In [28]:

print ("Best parameters: " , str(gbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(gbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'learning_rate': 0.025, 'loss': 'deviance', 'max_depth': 6, 'max_features': 'sqrt', 'min_impurity_decrease': 5, 'min_samples_leaf': 1, 'min_samples_split': 0.05, 'min_weight_fraction_leaf': 0.002, 'n_estimators': 100, 'subsample': 0.8}
Average score from cross-validation of best parameters on training data:  0.7781669846934232
F1 score of test data: 0.7738684736339154


In [29]:

parameters = {
    'learning_rate': [0.025, 0.05],
    'loss' : ['deviance'], 
    'n_estimators':[100],
    'subsample': [0.8],
    'criterion': ['friedman_mse'],
    'min_samples_split': [0.05],
    'min_samples_leaf': [1],
    'min_weight_fraction_leaf': [0.003, 0.002],
    'max_depth': [6],
    'min_impurity_decrease': np.arange(0, 5, 1),
    'max_features': ['sqrt'],
    'ccp_alpha': [0.0, 0.000001]
}
# n_cpus = multiprocessing.cpu_count()
gbc = GridSearchCV(gbclf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
gbc.fit(X_train,y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:   29.4s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:   32.1s finished


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'ccp_alpha': [0.0, 1e-06],
                         'criterion': ['friedman_mse'],
                         'learning_rate': [0.025, 0.05], 'loss': ['deviance'],
                         'max_depth': [6], 'max_features': ['sqrt'],
                         'min_impurity_decrease': array([0, 1, 2, 3, 4]),
                         'min_samples_leaf': [1], 'min_samples_split': [0.05],
                         'min_weight_fraction_leaf': [0.003, 0.002],
                         'n_estimators': [100], 'subsample': [0.8]},
             scoring='f1', verbose=1)

In [30]:

print ("Best parameters: " , str(gbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(gbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'ccp_alpha': 1e-06, 'criterion': 'friedman_mse', 'learning_rate': 0.025, 'loss': 'deviance', 'max_depth': 6, 'max_features': 'sqrt', 'min_impurity_decrease': 2, 'min_samples_leaf': 1, 'min_samples_split': 0.05, 'min_weight_fraction_leaf': 0.003, 'n_estimators': 100, 'subsample': 0.8}
Average score from cross-validation of best parameters on training data:  0.7784927753637546
F1 score of test data: 0.776220732874924


In [31]:

parameters = {
    'learning_rate': [0.025, 0.03, 0.02],
    'loss' : ['deviance'], 
    'n_estimators':[100],
    'subsample': [0.8],
    'criterion': ['friedman_mse'],
    'min_samples_split': [0.05],
    'min_samples_leaf': [1],
    'min_weight_fraction_leaf': [0.0025, 0.002, 0.0015],
    'max_depth': [6],
    'min_impurity_decrease': [1],
    'max_features': ['sqrt'],
    'ccp_alpha': [0.00001, 0.000001]
}
# n_cpus = multiprocessing.cpu_count()
gbc = GridSearchCV(gbclf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
gbc.fit(X_train,y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   16.4s finished


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'ccp_alpha': [1e-05, 1e-06],
                         'criterion': ['friedman_mse'],
                         'learning_rate': [0.025, 0.03, 0.02],
                         'loss': ['deviance'], 'max_depth': [6],
                         'max_features': ['sqrt'], 'min_impurity_decrease': [1],
                         'min_samples_leaf': [1], 'min_samples_split': [0.05],
                         'min_weight_fraction_leaf': [0.0025, 0.002, 0.0015],
                         'n_estimators': [100], 'subsample': [0.8]},
             scoring='f1', verbose=1)

In [32]:

print ("Best parameters: " , str(gbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(gbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'ccp_alpha': 1e-05, 'criterion': 'friedman_mse', 'learning_rate': 0.02, 'loss': 'deviance', 'max_depth': 6, 'max_features': 'sqrt', 'min_impurity_decrease': 1, 'min_samples_leaf': 1, 'min_samples_split': 0.05, 'min_weight_fraction_leaf': 0.0025, 'n_estimators': 100, 'subsample': 0.8}
Average score from cross-validation of best parameters on training data:  0.778714480892764
F1 score of test data: 0.775619906363794


In [33]:

parameters = {
    'learning_rate': [0.024, 0.025, 0.026],
    'loss' : ['deviance'], 
    'n_estimators':[100],
    'subsample': [0.8],
    'criterion': ['friedman_mse'],
    'min_samples_split': [0.05],
    'min_samples_leaf': [1],
    'min_weight_fraction_leaf': [0.0016, 0.0014, 0.0015],
    'max_depth': [6],
    'min_impurity_decrease': [1],
    'max_features': ['sqrt'],
    'ccp_alpha': [0.00001, 0.0001]
}
# n_cpus = multiprocessing.cpu_count()
gbc = GridSearchCV(gbclf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
gbc.fit(X_train,y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   17.2s finished


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'ccp_alpha': [1e-05, 0.0001],
                         'criterion': ['friedman_mse'],
                         'learning_rate': [0.024, 0.025, 0.026],
                         'loss': ['deviance'], 'max_depth': [6],
                         'max_features': ['sqrt'], 'min_impurity_decrease': [1],
                         'min_samples_leaf': [1], 'min_samples_split': [0.05],
                         'min_weight_fraction_leaf': [0.0016, 0.0014, 0.0015],
                         'n_estimators': [100], 'subsample': [0.8]},
             scoring='f1', verbose=1)

In [34]:

print ("Best parameters: " , str(gbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(gbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'ccp_alpha': 1e-05, 'criterion': 'friedman_mse', 'learning_rate': 0.025, 'loss': 'deviance', 'max_depth': 6, 'max_features': 'sqrt', 'min_impurity_decrease': 1, 'min_samples_leaf': 1, 'min_samples_split': 0.05, 'min_weight_fraction_leaf': 0.0016, 'n_estimators': 100, 'subsample': 0.8}
Average score from cross-validation of best parameters on training data:  0.7788392124087107
F1 score of test data: 0.7762906309751434


In [35]:

parameters = {
    'learning_rate': [0.024, 0.023, 0.021],
    'loss' : ['deviance'], 
    'n_estimators':[100],
    'subsample': [0.8, 0.79, 0.81],
    'criterion': ['friedman_mse'],
    'min_samples_split': [0.05, 0.049, 0.051],
    'min_samples_leaf': [1],
    'min_weight_fraction_leaf': [0.0016, 0.0014, 0.0015],
    'max_depth': [6],
    'min_impurity_decrease': [1, 0.999],
    'max_features': ['sqrt'],
    'ccp_alpha': [0.00011, 0.0001, 0.000999]
}
# n_cpus = multiprocessing.cpu_count()
gbc = GridSearchCV(gbclf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
gbc.fit(X_train,y_train)

Fitting 5 folds for each of 486 candidates, totalling 2430 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:   31.1s
[Parallel(n_jobs=-1)]: Done 430 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 780 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1230 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 1780 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 2430 out of 2430 | elapsed:  6.7min finished


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'ccp_alpha': [0.00011, 0.0001, 0.000999],
                         'criterion': ['friedman_mse'],
                         'learning_rate': [0.024, 0.023, 0.021],
                         'loss': ['deviance'], 'max_depth': [6],
                         'max_features': ['sqrt'],
                         'min_impurity_decrease': [1, 0.999],
                         'min_samples_leaf': [1],
                         'min_samples_split': [0.05, 0.049, 0.051],
                         'min_weight_fraction_leaf': [0.0016, 0.0014, 0.0015],
                         'n_estimators': [100],
                         'subsample': [0.8, 0.79, 0.81]},
             scoring='f1', verbose=1)

In [36]:

print ("Best parameters: " , str(gbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(gbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'ccp_alpha': 0.00011, 'criterion': 'friedman_mse', 'learning_rate': 0.023, 'loss': 'deviance', 'max_depth': 6, 'max_features': 'sqrt', 'min_impurity_decrease': 0.999, 'min_samples_leaf': 1, 'min_samples_split': 0.05, 'min_weight_fraction_leaf': 0.0015, 'n_estimators': 100, 'subsample': 0.79}
Average score from cross-validation of best parameters on training data:  0.7792480602406032
F1 score of test data: 0.7758321775312067


In [37]:

parameters = {
    'learning_rate': [0.0225, 0.023, 0.022],
    'loss' : ['deviance'], 
    'n_estimators':[100],
    'subsample': [0.82, 0.83, 0.81],
    'criterion': ['friedman_mse'],
    'min_samples_split': [0.05, 0.0499, 0.0501],
    'min_samples_leaf': [1],
    'min_weight_fraction_leaf': [0.00155, 0.00145, 0.0015],
    'max_depth': [6],
    'min_impurity_decrease': [0.9999, 0.999, 0.998],
    'max_features': ['sqrt'],
    'ccp_alpha': [0.000101, 0.0001, 0.0000999]
}
# n_cpus = multiprocessing.cpu_count()
gbc = GridSearchCV(gbclf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
gbc.fit(X_train,y_train)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:   31.7s
[Parallel(n_jobs=-1)]: Done 430 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 780 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1230 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 1780 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 2430 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 3180 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done 3645 out of 3645 | elapsed: 10.4min finished


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'ccp_alpha': [0.000101, 0.0001, 9.99e-05],
                         'criterion': ['friedman_mse'],
                         'learning_rate': [0.0225, 0.023, 0.022],
                         'loss': ['deviance'], 'max_depth': [6],
                         'max_features': ['sqrt'],
                         'min_impurity_decrease': [0.9999, 0.999, 0.998],
                         'min_samples_leaf': [1],
                         'min_samples_split': [0.05, 0.0499, 0.0501],
                         'min_weight_fraction_leaf': [0.00155, 0.00145, 0.0015],
                         'n_estimators': [100],
                         'subsample': [0.82, 0.83, 0.81]},
             scoring='f1', verbose=1)

In [38]:

print ("Best parameters: " , str(gbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(gbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'ccp_alpha': 0.0001, 'criterion': 'friedman_mse', 'learning_rate': 0.023, 'loss': 'deviance', 'max_depth': 6, 'max_features': 'sqrt', 'min_impurity_decrease': 0.9999, 'min_samples_leaf': 1, 'min_samples_split': 0.05, 'min_weight_fraction_leaf': 0.0015, 'n_estimators': 100, 'subsample': 0.82}
Average score from cross-validation of best parameters on training data:  0.7792536439945771
F1 score of test data: 0.7762541225481688


In [39]:

parameters = {
    'learning_rate': [0.0229, 0.023, 0.0231],
    'loss' : ['deviance'], 
    'n_estimators':[100],
    'subsample': [0.811, 0.809, 0.81],
    'criterion': ['friedman_mse'],
    'min_samples_split': [0.05, 0.04999, 0.05001],
    'min_samples_leaf': [1],
    'min_weight_fraction_leaf': [0.00155, 0.00156, 0.00154],
    'max_depth': [6],
    'min_impurity_decrease': [0.9999, 0.99999, 0.9998],
    'max_features': ['sqrt'],
    'ccp_alpha': [0.000101, 0.0001009, 0.000102]
}
# n_cpus = multiprocessing.cpu_count()
gbc = GridSearchCV(gbclf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
gbc.fit(X_train,y_train)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:   31.2s
[Parallel(n_jobs=-1)]: Done 430 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 780 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1230 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 1780 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 2430 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 3180 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done 3645 out of 3645 | elapsed: 10.2min finished


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'ccp_alpha': [0.000101, 0.0001009, 0.000102],
                         'criterion': ['friedman_mse'],
                         'learning_rate': [0.0229, 0.023, 0.0231],
                         'loss': ['deviance'], 'max_depth': [6],
                         'max_features': ['sqrt'],
                         'min_impurity_decrease': [0.9999, 0.99999, 0.9998],
                         'min_samples_leaf': [1],
                         'min_samples_split': [0.05, 0.04999, 0.05001],
                         'min_weight_fraction_leaf': [0.00155, 0.00156,
                                                      0.00154],
                         'n_estimators': [100],
                         'subsample': [0.811, 0.809, 0.81]},
             scoring='f1', verbose=1)

In [40]:

print ("Best parameters: " , str(gbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(gbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'ccp_alpha': 0.0001009, 'criterion': 'friedman_mse', 'learning_rate': 0.023, 'loss': 'deviance', 'max_depth': 6, 'max_features': 'sqrt', 'min_impurity_decrease': 0.9999, 'min_samples_leaf': 1, 'min_samples_split': 0.04999, 'min_weight_fraction_leaf': 0.00155, 'n_estimators': 100, 'subsample': 0.81}
Average score from cross-validation of best parameters on training data:  0.7792637804438203
F1 score of test data: 0.775829137002952


In [10]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
parameters = {
    'loss' : ['log_loss', 'auto', 'binary_cross_entropy', 'categorical_crossentropy'], 
}
gbhlf= HistGradientBoostingClassifier()
# n_cpus = multiprocessing.cpu_count()
ghbc = GridSearchCV(gbhlf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
ghbc.fit(X_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    2.4s finished


GridSearchCV(cv=5, estimator=HistGradientBoostingClassifier(), n_jobs=-1,
             param_grid={'loss': ['log_loss', 'auto', 'binary_cross_entropy',
                                  'categorical_crossentropy']},
             scoring='f1', verbose=1)

In [42]:

print ("Best parameters: " , str(ghbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(ghbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'loss': 'auto'}
Average score from cross-validation of best parameters on training data:  0.7768675451595116
F1 score of test data: 0.775829137002952


In [43]:
parameters = {
    'loss' : ['auto'], 
    'learning_rate': np.arange(0.0, 1.0, 0.1),
    'max_iter': [100, 1000],
    'max_leaf_nodes': [1, 31, 100, None]
}
# n_cpus = multiprocessing.cpu_count()
ghbc = GridSearchCV(gbhlf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
ghbc.fit(X_train,y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.3s
/Users/carolinejennings/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done 290 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:  1.3min finished


GridSearchCV(cv=5, estimator=HistGradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                         'loss': ['auto'], 'max_iter': [100, 1000],
                         'max_leaf_nodes': [1, 31, 100, None]},
             scoring='f1', verbose=1)

In [44]:

print ("Best parameters: " , str(ghbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(ghbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'learning_rate': 0.1, 'loss': 'auto', 'max_iter': 100, 'max_leaf_nodes': 31}
Average score from cross-validation of best parameters on training data:  0.7771954712005412
F1 score of test data: 0.775829137002952


In [45]:
parameters = {
    'loss' : ['auto'], 
    'learning_rate': [0.05, 0.1, 0.15],
    'max_iter': [100, 200],
    'max_leaf_nodes': [30, 31, 32],
    'max_depth': np.arange(1, 16, 2),
    'min_samples_leaf': np.arange(1, 101, 10),
    'max_bins': np.arange(100, 1001, 100),
}
# n_cpus = multiprocessing.cpu_count()
ghbc = GridSearchCV(gbhlf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
ghbc.fit(X_train,y_train)

Fitting 5 folds for each of 14400 candidates, totalling 72000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
/Users/carolinejennings/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 430 tasks      | elapsed:   30.2s
[Parallel(n_jobs=-1)]: Done 780 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1230 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 1780 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 2430 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 3180 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 4030 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 6400 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: 

GridSearchCV(cv=5, estimator=HistGradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1, 0.15], 'loss': ['auto'],
                         'max_bins': array([ 100,  200,  300,  400,  500,  600,  700,  800,  900, 1000]),
                         'max_depth': array([ 1,  3,  5,  7,  9, 11, 13, 15]),
                         'max_iter': [100, 200], 'max_leaf_nodes': [30, 31, 32],
                         'min_samples_leaf': array([ 1, 11, 21, 31, 41, 51, 61, 71, 81, 91])},
             scoring='f1', verbose=1)

In [46]:

print ("Best parameters: " , str(ghbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(ghbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'learning_rate': 0.05, 'loss': 'auto', 'max_bins': 200, 'max_depth': 9, 'max_iter': 100, 'max_leaf_nodes': 31, 'min_samples_leaf': 61}
Average score from cross-validation of best parameters on training data:  0.778847063444402
F1 score of test data: 0.775829137002952


In [47]:
parameters = {
    'loss' : ['auto'], 
    'learning_rate': [0.06, 0.05, 0.04],
    'max_iter': [90, 100, 110],
    'max_leaf_nodes': [31],
    'max_depth': np.arange(8, 9, 10),
    'min_samples_leaf': np.arange(1, 12, 2),
    'max_bins': np.arange(150, 301, 50),
}
# n_cpus = multiprocessing.cpu_count()
ghbc = GridSearchCV(gbhlf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
ghbc.fit(X_train,y_train)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-1)]: Done 643 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 1061 out of 1080 | elapsed:  1.6min remaining:    1.7s
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:  1.6min finished


GridSearchCV(cv=5, estimator=HistGradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.06, 0.05, 0.04], 'loss': ['auto'],
                         'max_bins': array([150, 200, 250, 300]),
                         'max_depth': array([8]), 'max_iter': [90, 100, 110],
                         'max_leaf_nodes': [31],
                         'min_samples_leaf': array([ 1,  3,  5,  7,  9, 11])},
             scoring='f1', verbose=1)

In [48]:

print ("Best parameters: " , str(ghbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(ghbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'learning_rate': 0.04, 'loss': 'auto', 'max_bins': 150, 'max_depth': 8, 'max_iter': 90, 'max_leaf_nodes': 31, 'min_samples_leaf': 7}
Average score from cross-validation of best parameters on training data:  0.7787957491022188
F1 score of test data: 0.775829137002952


In [49]:
parameters = {
    'loss' : ['auto'], 
    'learning_rate': [0.055, 0.05, 0.045],
    'max_iter': [99, 100, 101],
    'max_leaf_nodes': [31],
    'max_depth': [8, 9, 10],
    'min_samples_leaf': [6,7,8],
    'max_bins': np.arange(190, 261, 10),
}
# n_cpus = multiprocessing.cpu_count()
ghbc = GridSearchCV(gbhlf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
ghbc.fit(X_train,y_train)

Fitting 5 folds for each of 648 candidates, totalling 3240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:   21.0s
[Parallel(n_jobs=-1)]: Done 430 tasks      | elapsed:   51.0s
[Parallel(n_jobs=-1)]: Done 780 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 2000 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 2870 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 3221 out of 3240 | elapsed:  5.8min remaining:    2.1s
[Parallel(n_jobs=-1)]: Done 3240 out of 3240 | elapsed:  5.8min finished


GridSearchCV(cv=5, estimator=HistGradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.055, 0.05, 0.045],
                         'loss': ['auto'],
                         'max_bins': array([190, 200, 210, 220, 230, 240, 250, 260]),
                         'max_depth': [8, 9, 10], 'max_iter': [99, 100, 101],
                         'max_leaf_nodes': [31],
                         'min_samples_leaf': [6, 7, 8]},
             scoring='f1', verbose=1)

In [50]:

print ("Best parameters: " , str(ghbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(ghbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'learning_rate': 0.045, 'loss': 'auto', 'max_bins': 200, 'max_depth': 10, 'max_iter': 99, 'max_leaf_nodes': 31, 'min_samples_leaf': 7}
Average score from cross-validation of best parameters on training data:  0.779159308765023
F1 score of test data: 0.775829137002952


In [53]:
# print(df.head())
# categorical_columns_subset = [
#     "HeartDisease",
#     "Smoking",
#     "AlcoholDrinking",
#     "Stroke",
#     "Sex",
#     "AgeCategory",
#     "Race",
#     "PhysicalActivity",
#     "GenHealth",
#     "Asthma",
#     "KidneyDisease",
#     "SkinCancer"
    
# ]

# numerical_columns_subset = [
#     "BMI",
#     "PhysicalHealth",
#     "MentalHealth",
#     "DiffWalking",
#     "SleepTime"
# ]

# Mymy = df[categorical_columns_subset + numerical_columns_subset]
# Mymy[categorical_columns_subset] = Mymy[categorical_columns_subset].astype("category")

# n_categorical_features = Mymy.select_dtypes(include="category").shape[1]
# n_numerical_features = Mymy.select_dtypes(include="number").shape[1]

# categorical_mask = [True] * n_categorical_features + [False] * n_numerical_features
parameters = {
    'loss' : ['auto'], 
    'learning_rate': [0.044, 0.046, 0.045],
    'max_iter': [98, 99],
    'max_leaf_nodes': [31],
    'max_depth': [9],
    'min_samples_leaf': [8, 9],
    'max_bins': np.arange(220, 241, 5)
#     'categorical_features': [categorical_mask, None]
}

# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import OrdinalEncoder
# from sklearn.compose import make_column_transformer

# ordinal_encoder = make_column_transformer(
#     (
#         OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan),
#         make_column_selector(dtype_include="category"),
#     ),
#     remainder="passthrough",
# )

# # n_cpus = multiprocessing.cpu_count()
# hist_native = make_pipeline(
#     ordinal_encoder,
#     HistGradientBoostingRegressor(categorical_features=categorical_mask)
# )
# gbhlf =hist_native

ghbc = GridSearchCV(gbhlf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
ghbc.fit(X_train,y_train)


Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   36.1s finished


GridSearchCV(cv=5, estimator=HistGradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.044, 0.046, 0.045],
                         'loss': ['auto'],
                         'max_bins': array([220, 225, 230, 235, 240]),
                         'max_depth': [9], 'max_iter': [98, 99],
                         'max_leaf_nodes': [31], 'min_samples_leaf': [8, 9]},
             scoring='f1', verbose=1)

In [55]:
df.head()
gbhlf.get_params().keys()

print ("Best parameters: " , str(ghbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(ghbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'learning_rate': 0.044, 'loss': 'auto', 'max_bins': 220, 'max_depth': 9, 'max_iter': 98, 'max_leaf_nodes': 31, 'min_samples_leaf': 8}
Average score from cross-validation of best parameters on training data:  0.7791347786093038
F1 score of test data: 0.775829137002952


In [56]:
parameters = {
    'loss' : ['auto'], 
    'learning_rate': [0.044, 0.0441, 0.0439],
    'max_iter': [98, 97],
    'max_leaf_nodes': [31],
    'max_depth': [9],
    'min_samples_leaf': [10, 9],
    'max_bins': np.arange(215, 233, 2)
#     'categorical_features': [categorical_mask, None]
}


ghbc = GridSearchCV(gbhlf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
ghbc.fit(X_train,y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:   24.1s
[Parallel(n_jobs=-1)]: Done 430 tasks      | elapsed:   54.8s
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  1.1min finished


GridSearchCV(cv=5, estimator=HistGradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.044, 0.0441, 0.0439],
                         'loss': ['auto'],
                         'max_bins': array([215, 217, 219, 221, 223, 225, 227, 229, 231]),
                         'max_depth': [9], 'max_iter': [98, 97],
                         'max_leaf_nodes': [31], 'min_samples_leaf': [10, 9]},
             scoring='f1', verbose=1)

In [57]:

print ("Best parameters: " , str(ghbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(ghbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'learning_rate': 0.044, 'loss': 'auto', 'max_bins': 217, 'max_depth': 9, 'max_iter': 98, 'max_leaf_nodes': 31, 'min_samples_leaf': 10}
Average score from cross-validation of best parameters on training data:  0.7787569865610772
F1 score of test data: 0.775829137002952


In [60]:
parameters = {
    'loss' : ['auto'], 
    'learning_rate': [0.044, 0.04401, 0.04399],
    'max_iter': [98],
    'max_leaf_nodes': [31],
    'max_depth': [9],
    'min_samples_leaf': [10, 11],
    'max_bins': np.arange(215, 222, 1)
#     'categorical_features': [categorical_mask, None]
}


ghbc = GridSearchCV(gbhlf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
ghbc.fit(X_train,y_train)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:   22.4s
[Parallel(n_jobs=-1)]: Done 210 out of 210 | elapsed:   26.0s finished


GridSearchCV(cv=5, estimator=HistGradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.044, 0.04401, 0.04399],
                         'loss': ['auto'],
                         'max_bins': array([215, 216, 217, 218, 219, 220, 221]),
                         'max_depth': [9], 'max_iter': [98],
                         'max_leaf_nodes': [31], 'min_samples_leaf': [10, 11]},
             scoring='f1', verbose=1)

In [61]:

print ("Best parameters: " , str(ghbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(ghbc.best_score_))
print("F1 score of test data:", gbc.score(X_test, y_test))

Best parameters:  {'learning_rate': 0.04401, 'loss': 'auto', 'max_bins': 216, 'max_depth': 9, 'max_iter': 98, 'max_leaf_nodes': 31, 'min_samples_leaf': 10}
Average score from cross-validation of best parameters on training data:  0.7786692820553384
F1 score of test data: 0.775829137002952


In [14]:
parameters = {
    'loss' : ['auto'], 
    'learning_rate': [0.04402, 0.04401, 0.0440099],
    'max_iter': [98],
    'max_leaf_nodes': [31],
    'max_depth': [9],
    'min_samples_leaf': [10],
    'max_bins': [216]
#     'categorical_features': [categorical_mask, None]
}


ghbc = GridSearchCV(gbhlf, parameters,cv=5, scoring='f1', verbose=1, n_jobs=-1)
ghbc.fit(X_train,y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:    2.6s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    2.7s finished


GridSearchCV(cv=5, estimator=HistGradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.04402, 0.04401, 0.0440099],
                         'loss': ['auto'], 'max_bins': [216], 'max_depth': [9],
                         'max_iter': [98], 'max_leaf_nodes': [31],
                         'min_samples_leaf': [10]},
             scoring='f1', verbose=1)

In [15]:

print ("Best parameters: " , str(ghbc.best_params_))
print("Average score from cross-validation of best parameters on training data: ", str(ghbc.best_score_))
print("F1 score of test data:", ghbc.score(X_test, y_test))

Best parameters:  {'learning_rate': 0.0440099, 'loss': 'auto', 'max_bins': 216, 'max_depth': 9, 'max_iter': 98, 'max_leaf_nodes': 31, 'min_samples_leaf': 10}
Average score from cross-validation of best parameters on training data:  0.7738244322571978
F1 score of test data: 0.7780504908835906
